In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

etas = [1, 3, 5, 7, 9]
results_dir = Path("Results")
methods = ['GMM', 'EW', 'MC', 'DRMC', 'CMC', 'DRCMC', 'OTCMC']
tau = 0.1
ANN_DAYS = 252

def summarize_file(fpath: Path, eta: int, methods, tau=0.1):

    df = pd.read_csv(fpath)
    if 'Trial' in df.columns:
        df = df[df['Trial'] != 'AVG']

    rows = []
    for m in methods:
        mean_col = f"mean_{m}"
        loss_col = f"loss_{m}"

        returns = pd.to_numeric(df[mean_col], errors='coerce').dropna()
        losses  = pd.to_numeric(df[loss_col], errors='coerce').dropna()

        mu = returns.mean()
        sd = returns.std(ddof=1)
        sharpe = mu / sd if sd != 0 else np.nan
        sharpe_ann = sharpe * np.sqrt(ANN_DAYS) if np.isfinite(sharpe) else np.nan
        exp_loss = losses.mean()

        sorted_ret = returns.sort_values()
        k = int(len(sorted_ret) * tau)
        cvar_ret = sorted_ret.iloc[:k].mean() if k > 0 else np.nan
        cvar_loss = -cvar_ret if k > 0 else np.nan

        rows.append({
            "eta": eta,
            "method": m,
            "exp.loss": exp_loss,
            "mean": mu,
            f"C{tau}": cvar_loss,
            "Sharpe": sharpe_ann
        })
    return rows

all_rows = []
for eta in etas:
    f = results_dir / f"PF_full_eta{eta}.csv"
    all_rows.extend(summarize_file(f, eta, methods, tau=tau))

result_df = pd.DataFrame(all_rows)
result_df = result_df.round(3)

ranked = (result_df.sort_values(["eta", "Sharpe"], ascending=[True, False])
                    .reset_index(drop=True))

print("=== Combined summary (all eta) ===")
print(ranked.to_string(index=False))

pivot_sharpe = (result_df.pivot_table(index="method", columns="eta", values="Sharpe", aggfunc="first")
                           .round(3))
pivot_loss   = (result_df.pivot_table(index="method", columns="eta", values="exp.loss", aggfunc="first")
                           .round(3))
pivot_cvar   = (result_df.pivot_table(index="method", columns="eta", values=f"C{tau}", aggfunc="first")
                           .round(3))

print("\n=== Sharpe (annualized) by eta ===")
print(pivot_sharpe.to_string())

print("\n=== Expected loss by eta ===")
print(pivot_loss.to_string())

print(f"\n=== CVaR (loss, tau={tau}) by eta ===")
print(pivot_cvar.to_string())


=== Combined summary (all eta) ===
 eta method  exp.loss  mean  C0.1  Sharpe
   1   DRMC    -0.222 0.111 1.371   2.301
   1    GMM    -0.236 0.118 1.476   2.240
   1  OTCMC    -0.236 0.118 1.492   2.205
   1  DRCMC    -0.237 0.118 1.498   2.202
   1     EW    -0.236 0.118 1.501   2.196
   1     MC    -0.115 0.058 1.140   1.366
   1    CMC    -0.098 0.049 2.207   0.691
   3     MC    -0.457 0.114 1.282   2.427
   3   DRMC    -0.451 0.113 1.407   2.263
   3    GMM    -0.481 0.120 1.492   2.251
   3  OTCMC    -0.469 0.117 1.485   2.208
   3  DRCMC    -0.473 0.118 1.498   2.202
   3     EW    -0.472 0.118 1.501   2.196
   3    CMC    -0.051 0.013 2.931   0.129
   5    GMM    -0.730 0.122 1.503   2.256
   5   DRMC    -0.683 0.114 1.432   2.241
   5  DRCMC    -0.710 0.118 1.498   2.202
   5  OTCMC    -0.702 0.117 1.486   2.201
   5     EW    -0.709 0.118 1.501   2.196
   5     MC    -0.670 0.112 1.412   2.188
   5    CMC    -0.357 0.059 2.809   0.657
   7    GMM    -0.979 0.122 1.509   2.257